# MEDCouplingRemapper: interpolation of fields

Here, we will perform an interpolation between two meshes `srcMesh` and `trgMesh`. To emphasize some subtleties of interpolation, we take a particular case where `srcMesh` is a refined mesh of `trgMesh` (with some cells cut more finely).

To start the exercise, import the `medcoupling` module and the `MEDCouplingRemapper` class from the module.

In [1]:
import medcoupling as mc
from medcoupling import MEDCouplingRemapper

## Create the target mesh

Construct the unstructured mesh `trgMesh` from a 2D Cartesian mesh 10x10 starting at point `[0.,0.]` and having a step of 1 in both X and Y:

In [2]:
arr = mc.DataArrayDouble(11)
arr.iota(0)
trgMesh = mc.MEDCouplingCMesh()
trgMesh.setCoords(arr, arr)
trgMesh = trgMesh.buildUnstructured()

## Create the source mesh

Create a mesh `srcMesh` from a 2D Cartesian mesh of 20x20 cells also starting at point `[0.,0.]` and having a step of 0.5 in both X and Y:

In [3]:
arr = mc.DataArrayDouble(21)
arr.iota(0)
arr *= 0.5
srcMesh = mc.MEDCouplingCMesh()
srcMesh.setCoords(arr, arr)
srcMesh = srcMesh.buildUnstructured()

To make the exercise more interesting, triangulate the first 20 cells of `srcMesh` using `MEDCouplingUMesh.simplexize()` (2D simplices are triangles). Set the result to `srcMesh`.

In [4]:
tmp = srcMesh[:20]  # Extract a sub-part of srcMesh
tmp.simplexize(0)
srcMesh = mc.MEDCouplingUMesh.MergeUMeshes([tmp, srcMesh[20:]])

Interpolate with MEDCouplingRemapper

We recall that to project a field from one mesh to another, we must first prepare the interpolation matrix containing the projection ratios.

Calculate the first part of the interpolation matrix from `srcMesh` (discretized at cells - P0) to `trgMesh` (also discretized at cells). To do this, invoke `MEDCouplingRemapper.prepare()` on an instance (`remap`) of `MEDCouplingRemapper`.

In [5]:
remap = MEDCouplingRemapper()
remap.prepare(srcMesh, trgMesh, "P0P0")

1

Check that the matrix calculated by the method is correct in our trivial case. To do this, retrieve in `myMatrix` the internal matrix returned by `MEDCouplingRemapper.getCrudeMatrix()`. This gives for each cell of `trgMesh` the identifiers of cells of `srcMesh` with which it intersects, and the corresponding intersection area.

Make sure that for each cell of `trgMesh`, the sum of the areas always equals 1.

In [6]:
myMatrix = remap.getCrudeMatrix()
print(myMatrix)
sumByRows = mc.DataArrayDouble(len(myMatrix))
for i, wIt in enumerate(sumByRows):
    su = 0.0
    for it in myMatrix[i]:
        su += myMatrix[i][it]
    wIt[0] = su
print("Is interpolation well prepared?", sumByRows.isUniform(1.0, 1e-12))

[{0: 0.125, 1: 0.125, 2: 0.125, 3: 0.125, 40: 0.25, 41: 0.25}, {4: 0.125, 5: 0.125, 6: 0.125, 7: 0.125, 42: 0.25, 43: 0.25}, {8: 0.125, 9: 0.125, 10: 0.125, 11: 0.125, 44: 0.25, 45: 0.25}, {12: 0.125, 13: 0.125, 14: 0.125, 15: 0.125, 46: 0.25, 47: 0.25}, {16: 0.125, 17: 0.125, 18: 0.125, 19: 0.125, 48: 0.25, 49: 0.25}, {20: 0.125, 21: 0.125, 22: 0.125, 23: 0.125, 50: 0.25, 51: 0.25}, {24: 0.125, 25: 0.125, 26: 0.125, 27: 0.125, 52: 0.25, 53: 0.25}, {28: 0.125, 29: 0.125, 30: 0.125, 31: 0.125, 54: 0.25, 55: 0.25}, {32: 0.125, 33: 0.125, 34: 0.125, 35: 0.125, 56: 0.25, 57: 0.25}, {36: 0.125, 37: 0.125, 38: 0.125, 39: 0.125, 58: 0.25, 59: 0.25}, {60: 0.25, 61: 0.25, 80: 0.25, 81: 0.25}, {62: 0.25, 63: 0.25, 82: 0.25, 83: 0.25}, {64: 0.25, 65: 0.25, 84: 0.25, 85: 0.25}, {66: 0.25, 67: 0.25, 86: 0.25, 87: 0.25}, {68: 0.25, 69: 0.25, 88: 0.25, 89: 0.25}, {70: 0.25, 71: 0.25, 90: 0.25, 91: 0.25}, {72: 0.25, 73: 0.25, 92: 0.25, 93: 0.25}, {74: 0.25, 75: 0.25, 94: 0.25, 95: 0.25}, {76: 0.25, 77

Construct a field on cells "srcField" built from the following analytical formula: `7-sqrt((x-5.)*(x-5.)+(y-5.)*(y-5.))`:

In [7]:
srcField = mc.MEDCouplingFieldDouble(mc.ON_CELLS, mc.ONE_TIME)
srcField.setMesh(srcMesh)
srcField.fillFromAnalytic(1, "7-sqrt((x-5.)*(x-5.)+(y-5.)*(y-5.))")
srcField.getArray().setInfoOnComponent(0, "powercell [W]")

Here is what this field looks like:

<img src="Remapper1.png" style="width:500px;">

Apply interpolation with `MEDCouplingRemapper.transferField()`:

In [8]:
# remap.transferField(srcField, 1e300)

<div class="alert alert-block alert-success">
<b>Note:</b> 1e300 is a default value. This value will be systematically assigned to any cell of trgField not intersecting any cell of srcMesh. Typically, users set an enormous value to spot what is often a bug. However, other users, from the perspective of parallel interpolation for example, set 0.</div>

<div class="alert alert-block alert-success">
<b>Note:</b> An exception is thrown because srcField has no defined nature. We will see later in the impact of this attribute on the final result.</div>

Set the nature of `srcField` to `IntensiveMaximum`. This means that the field should be interpreted as intensive (such as temperature).

In [9]:
srcField.setNature(mc.IntensiveMaximum)
trgFieldCV = remap.transferField(srcField, 1e300)

Check that with the `IntensiveMaximum` nature, the integral of the field is preserved. However, the sum over the cells (accumulation) is not preserved!

In [10]:
integSource = srcField.integral(True)[0]
integTarget = trgFieldCV.integral(True)[0]
print("IntensiveMaximum -- integrals: %lf == %lf" % (integSource, integTarget))

accSource = srcField.getArray().accumulate()[0]
accTarget = trgFieldCV.getArray().accumulate()[0]
print("IntensiveMaximum -- sums: %lf != %lf" % (accSource, accTarget))

IntensiveMaximum -- integrals: 317.757888 == 317.757888
IntensiveMaximum -- sums: 1300.653980 != 317.757888


Now set the nature of `srcField` to `ExtensiveConservation`. The field should be interpreted as extensive (such as power or volume).

In [11]:
srcField.setNature(mc.ExtensiveConservation)
trgFieldI = remap.transferField(srcField, 1e300)

Check that with the `ExtensiveConservation` nature, the integral of the field is not preserved. However, the sum over the cells is preserved.

In [12]:
integSource = srcField.integral(True)[0]
integTarget = trgFieldI.integral(True)[0]
print("ExtensiveConservation -- integrals: %lf != %lf" % (integSource, integTarget))

accSource = srcField.getArray().accumulate()[0]
accTarget = trgFieldI.getArray().accumulate()[0]
print("ExtensiveConservation -- sums: %lf == %lf" % (accSource, accTarget))

ExtensiveConservation -- integrals: 317.757888 != 1300.653980
ExtensiveConservation -- sums: 1300.653980 == 1300.653980


Visualize the fields with ParaView, or by writing them to a file.